# Build GPT part 1

### I. Load raw data

In [5]:
# We always start with a dataset to train on. 
# Let's download the tiny shakespeare dataset

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-31 13:34:04--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  6.87MB/s    in 0.2s    

2023-07-31 13:34:05 (6.87 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [6]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [7]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [10]:
# let's look at the first 1000 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [11]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### 2. Tokenize

In [13]:
# create a simple tokenizer
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


**Note** Tokenization methods to try (sub-words is usually chosen):
- try senterpiece by google
- try tiktoken from openai

In [16]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:100])

torch.Size([1115394]) <built-in method type of Tensor object at 0x16479b090>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


### 3. Inputs

In [17]:
# split the data

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [18]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

**dim1**: We have 8 different "hidden" examples in each x/y pair.

In [19]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


**dim2**: The next dimension of our input is each **batch**

In [43]:
torch.manual_seed(1337)

batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x 
    # and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(
        low=0, 
        high=len(data) - block_size, 
        size=(batch_size,))

    # get the context for each index, and stack it into
    # rows.
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

**Example**

In [44]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
        

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

This is our **input** to the transformer

In [70]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [62]:
yb

tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

### 4. Baseline Model: Bi-grams 

In [119]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        # each token directly reads off the logits for the next token 
        # from a look-up table.
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        # remember we have 8 examples for each row in X
        
        # reminder: (B, T, C) = (batch, time, channel)
        logits = self.token_embedding_table(idx) # -> (B, T, C)

        if targets is None:
            loss = None
        else:
            # cross entropy requires a (B, C, T) shape
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
    
            # focus only on the last time step 
            # i.e. last token in sequence
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [133]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([256, 65])
tensor(4.5071, grad_fn=<NllLossBackward0>)


In [134]:
# we expect a logloss of -ln(1/65) = 4.174
# we got a logloss of 4.876, meaning that theres is entropy across our
# initial values (i.e. not uniform distribution)

In [135]:
vocab_size

65

In [136]:
xb

tensor([[58,  1, 44, 47, 56, 57, 58,  6],
        [46, 53, 59,  1, 39, 56, 58,  1],
        [50, 43, 43, 42, 47, 52, 45,  6],
        [46, 43,  1, 61, 39, 57,  1, 50],
        [56, 47, 58, 39, 40, 50, 43,  1],
        [52, 42,  1, 58, 46, 43, 56, 43],
        [50, 43, 51, 39, 52, 11,  0, 13],
        [43,  1, 56, 43, 57, 58, 53, 56],
        [31, 10,  0, 32, 46, 47, 57,  1],
        [32, 21, 27, 10,  0, 25, 53, 56],
        [53, 46, 52,  1, 53, 44,  1, 19],
        [44, 39, 56,  1, 58, 46, 39, 58],
        [30, 17, 26, 15, 17, 10,  0, 32],
        [51, 43,  6,  1, 39, 52, 42,  1],
        [56,  8,  0,  0, 19, 24, 27, 33],
        [47, 52,  1, 58, 46, 47, 57, 12],
        [50, 50,  1, 50, 43, 58,  1, 58],
        [43, 43,  2,  0,  0, 19, 24, 27],
        [44,  1, 39,  1, 40, 39, 61, 42],
        [17, 42, 61, 39, 56, 42,  6,  0],
        [ 1, 39, 50, 50,  1, 51, 63,  1],
        [ 1, 21,  1, 42, 53,  1, 56, 43],
        [39, 47, 42,  1, 61, 43, 50, 50],
        [52, 45,  0, 35, 46, 43, 5

In [137]:
yb

tensor([[ 1, 44, 47, 56, 57, 58,  6,  1],
        [53, 59,  1, 39, 56, 58,  1, 54],
        [43, 43, 42, 47, 52, 45,  6,  1],
        [43,  1, 61, 39, 57,  1, 50, 47],
        [47, 58, 39, 40, 50, 43,  1, 42],
        [42,  1, 58, 46, 43, 56, 43,  1],
        [43, 51, 39, 52, 11,  0, 13, 52],
        [ 1, 56, 43, 57, 58, 53, 56, 43],
        [10,  0, 32, 46, 47, 57,  1, 42],
        [21, 27, 10,  0, 25, 53, 56, 43],
        [46, 52,  1, 53, 44,  1, 19, 39],
        [39, 56,  1, 58, 46, 39, 58,  1],
        [17, 26, 15, 17, 10,  0, 32, 46],
        [43,  6,  1, 39, 52, 42,  1, 58],
        [ 8,  0,  0, 19, 24, 27, 33, 15],
        [52,  1, 58, 46, 47, 57, 12,  0],
        [50,  1, 50, 43, 58,  1, 58, 46],
        [43,  2,  0,  0, 19, 24, 27, 33],
        [ 1, 39,  1, 40, 39, 61, 42, 10],
        [42, 61, 39, 56, 42,  6,  0, 37],
        [39, 50, 50,  1, 51, 63,  1, 46],
        [21,  1, 42, 53,  1, 56, 43, 44],
        [47, 42,  1, 61, 43, 50, 50, 12],
        [45,  0, 35, 46, 43, 56, 4

In [138]:
# generate an output
idx = torch.zeros((1,1), dtype=torch.long) # (B, T)

# look at first item in batch
decode(m.generate(idx, max_new_tokens=100)[0].tolist())

"\nSGu \nxXT.cksv\nK;EhGw'AWhwbcSzIQEYwBbcZEAPZH'x?B!c;QJjPvpKT3Tb-wb:NuFyoUW\nKhmM:TYxSfB&Cwlhb!r.nFhuie&"

### 5. Train

In [139]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [140]:
# training loop
batch_size = 32

for steps in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())    

2.3780767917633057


In [146]:
# generate an output
idx = torch.zeros((1,1), dtype=torch.long) # (B, T)

# look at first item in batch
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))



K:

AULUGLu yeringreadedon, t ch r serosbloichy f me! p mis. thas Se.
Witourans ue d cthemad,

EScou.
G med weratt,
TUCKETht, merisurthaIARe hioune dwercoe,
TCIOUSou, in adl.
The my seaves merro we man byoowerejWeel ber titd ore y f covone s d:
Wruge ace s quld hayofourerost hary;
ossthathe KI bungrs it
Weofs to bup t.
Onte to wangotulleentulil me fuHAngrt he wem,
Wind, fe d lthas as norrl ad bh he; kit myowellell LOf d t gnghorthame mace:

Touinsoudonthtoopastrsthithound Tham.
Fiom no s; fat i
